# Load Libraries

In [1]:
import numpy as np
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Main Functions

In [2]:
def load_data(file):
    # Load  data from file and return it as panads dataframe
    return pd.read_csv(file)

def scale_features(X_train, X_test, scaler='standard'):
    # load x data and returned it as a scaled data
    if scaler == 'minmax':
        scaler = MinMaxScaler()
    else:
        scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled        


# NLP Functions

In [3]:
# Drop Null Rows
def drop_null(train, test):
    train = train.dropna()
    test  = test.dropna().reset_index(drop=True)    
    return train, test

In [4]:
# Remove Non-Arabic Letters
def filter_arabic(text):
    import re
    arabic_pattern = re.compile(r'[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF]+')
    arabic_text = arabic_pattern.findall(text)
    return ' '.join(arabic_text)

In [5]:
# Remove Dublicate Questions
def remove_duplicates_set(dataset):
  unique_questions = []
  seen_questions = {}
  for col in dataset:
    if col['question'] not in seen_questions:
      seen_questions[question] = True
    else:
      dataset.drop(axis=0,inplace=True)  
  return dataset

In [6]:
# Clean text using regular expression
def cleanText(text):
  import re
  text = re.sub(r'@|#\w+', '', text) # Remove hashtags or mentions
  text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text) # Substitute all URLs in the text with an empty string
  text = re.sub(r'(\?|!)+', '', text) # Remove question mark? And another one with an exclamation mark!
  text = re.sub(r'\d+', '', text) # Remove Numbers
  text = re.sub(r'[^\w\s]', '', text) # Remove punctuation
  text = re.sub(r'^\s+', '', text) # Remove leading spaces
  text = re.sub(r'\s+$', '', text) # Remove trailing spaces
  return text

In [7]:
# Stop Words
from nltk.corpus import stopwords
nltk.download('stopwords')
arabic_stopwords = stopwords.words('arabic')

def remove_stop_words(text):
  return ' '.join([word for word in text.split() if word not in arabic_stopwords])


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\OMAR\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# Normalizes Arabic letters in a text by replacing different forms of the latter (أ, آ, إ) to (ا) for example
import pyarabic.araby as araby
def normalize_arabic_letters(text):
    text = araby.strip_tashkeel(text)
    text = araby.strip_diacritics(text)
    text = araby.strip_shadda(text)
    text = araby.normalize_hamza(text,method="tasheel")
    return text

In [9]:
from nltk import word_tokenize
def count_arabic_words(data):
  df = pd.DataFrame({})
  def count_words(text):
    words = word_tokenize(text)  # Tokenize
    return len(words) 
   
  df['question'] = data['question']
  df['answer'] = data['answer']
  df['question_counter'] = data['question'].apply(count_words)
  df['answer_counter']   = data['answer'].apply(count_words)
  df['label']   = data['label']
  return df

In [94]:
# Generate Vocabularies List
from pyarabic.araby import tokenize
def generateVocabulary(data):
    words = tokenize(data)
    return words     


# Model

## Load & Describe Data

In [100]:
train_data = load_data('train.csv')
test_data  = load_data('val.csv')

## PreProcessing DataSet

In [101]:
# 1.Drop Null Data From Train and Test Data   
train , test = drop_null(train_data, test_data)

In [102]:
# 2. Filtering Non-Arabic Content: Remove any irrelevant content that is not in Arabic
train.loc[:, 'question']  = train['question'].apply(filter_arabic)
train.loc[:, 'answer'  ]  = train['answer'].apply(filter_arabic)
test.loc[: , 'question']  = test['question'].apply(filter_arabic)
test.loc[: , 'answer'  ]  = test['answer'].apply(filter_arabic)

In [103]:
# 3. Remove duplicated questions  from dataset 
train_unique_data = train.drop_duplicates(subset=['question'])
test_unique_data  = test.drop_duplicates(subset=['question'])


In [104]:
# 4 Clean Text from sprecail cahr and so on
train_unique_data.loc[:, 'question'] = train_unique_data['question'].apply(cleanText)
train_unique_data.loc[:, 'answer'  ] = train_unique_data['answer'].apply(cleanText)
test_unique_data.loc[: , 'question'] = test_unique_data['question'].apply(cleanText)
test_unique_data.loc[: , 'answer'  ] = test_unique_data['answer'].apply(cleanText)

In [105]:
# 6 Normalize arabic letters
train_unique_data.loc[:, 'question'] = train_unique_data['question'].apply(normalize_arabic_letters)
train_unique_data.loc[:, 'answer'  ] = train_unique_data['answer'].apply(normalize_arabic_letters)
test_unique_data.loc[: , 'question'] = test_unique_data['question'].apply(normalize_arabic_letters)
test_unique_data.loc[: , 'answer'  ] = test_unique_data['answer'].apply(normalize_arabic_letters)

## Building Model

In [106]:
train_unique_data.head()

,question,answer,label
0,ما هي مميزات و عيوب الدواء جلوكوفانس و ايضا ال...,لكل علاج ايجابيته وسلبياته والتي تعتمد على حال...,الدم
1,اليك نتيجة تحليل هرمونات الغدة الدرقية علما با...,نعم يجب تخفيض الجرعة الا اذا كان سبب استيصال ا...,الاورام-الخبيثة-والحميدة
2,حلول منزلية لاعراض ارتفاع ضغط الدم,يفضل عدم الاستغناء عن العلاج الدوايي لمرضى الض...,جراحة-عامة
3,عملت عملية دوالي الساقين قسطرة الليزر من شهر و...,راجع طبيبك من اجري الجراحه افضل من يجيب لانه ش...,أمراض-الجهاز-التنفسي
4,ما حقيقة ان تمرين العضلة النعلية يخفض السكر بالدم,اذا قصدت تدليك العضلة فهذا كلام غير صحيح ولكن ...,مرض-السكري


In [107]:
train_unique_data.loc[:,'arabic_tokens'] = train_unique_data['answer'].apply(generateVocabulary)


C:\Users\OMAR\AppData\Local\Temp\ipykernel_9996\329683929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_unique_data.loc[:,'arabic_tokens'] = train_unique_data['answer'].apply(generateVocabulary)


In [121]:
train_unique_data.tail(20)

,question,answer,label,arabic_tokens
52725,ماهو علاج مرض ذيل الثدي,ذيل الثدي ليس مرض وهو جزء من الثدي نفسه ولا يح...,الاورام-الخبيثة-والحميدة,"[ذيل, الثدي, ليس, مرض, وهو, جزء, من, الثدي, نف..."
52728,السلام عليكم اعانى من ولرم ليفى فى ثدي اريد مع...,الورم الليفي في الثدي حميد ولا داعي لاستيصاله ...,الاورام-الخبيثة-والحميدة,"[الورم, الليفي, في, الثدي, حميد, ولا, داعي, لا..."
52729,انا كان عندي زي نقطه سودا ناحيه صدري ف هرشتها ...,يجب الفحص من قبل طبيب جراح للوصول لتشخيص واستب...,جراحة-عامة,"[يجب, الفحص, من, قبل, طبيب, جراح, للوصول, لتشخ..."
52730,عمتي كبيره ف السن تعاني من عدم النوم وتوهم حاج...,ننصحك بمراجعة طبيب امراض نفسية ضخامة الغدة تحت...,امراض-الغدد-الصماء,"[ننصحك, بمراجعة, طبيب, امراض, نفسية, ضخامة, ال..."
52731,انا بعاني من النحافة الزايدة وقبل سنتين صار مع...,السلام عليكم اذا عندك فرط في نشاط العده تحتاج ...,امراض-الغدد-الصماء,"[السلام, عليكم, اذا, عندك, فرط, في, نشاط, العد..."
52732,لدي صعوبة التنفس واعطس كثيرا و كثيرا ما اصاب ب...,لا دواء قبل مراجعة اخصايي الامراض الصدرية والح...,امراض-الجهاز-التنفسي,"[لا, دواء, قبل, مراجعة, اخصايي, الامراض, الصدر..."
52735,اعاني من الام في الجانب الايسر في منطقة الثدي ...,الالم الذي تصف به هو الم عضلي يزول بمضادات الا...,امراض-الجهاز-التنفسي,"[الالم, الذي, تصف, به, هو, الم, عضلي, يزول, بم..."
52736,الناسور بتشاف بالعين,في بعض الاحيان يكون سهل رويته ولكن ليس دايما ا...,جراحة-عامة,"[في, بعض, الاحيان, يكون, سهل, رويته, ولكن, ليس..."
52737,كان عندي سكر حمل ولدت واختفى السكر حللت صايم ب...,عادة يختفي مرض السكر بعد الولادة لفترة تعتمد ع...,امراض-الغدد-الصماء,"[عادة, يختفي, مرض, السكر, بعد, الولادة, لفترة,..."
52738,هل هناك علاقه او سيستفاد منها الطبيب المعالج ل...,نعم يوجد هناك علاقة وسوف تفيد في التوجه نحو خط...,ارتفاع-ضغط-الدم,"[نعم, يوجد, هناك, علاقة, وسوف, تفيد, في, التوج..."


In [119]:
vocs = train_unique_data['arabic_tokens']
from collections import Counter
import nltk
nltk.download('wordpunct_tokenizer')
series = vocs.str.lower()
tokens = series.apply(lambda x: nltk.wordpunct_tokenizer.tokenize(x))


[nltk_data] Error loading wordpunct_tokenizer: Package
[nltk_data]     'wordpunct_tokenizer' not found in index


AttributeError: module 'nltk' has no attribute 'wordpunct_tokenizer'

In [115]:
unique_words

[]

In [110]:
unique_words

[]